In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn 
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from tqdm import tqdm

In [356]:
df = pd.read_csv("../Data/single_dataset.csv", low_memory=False)
df.dropna(axis=0, how='any', inplace=True)

#enc characters
enc_p1 = LabelEncoder()
enc_p2 = LabelEncoder()

enc_p1.fit(df['CHAR_P1'])
enc_p2.fit(df['CHAR_P2'])
df['CHAR_P1'] = enc_p1.transform(df['CHAR_P1'])
df['CHAR_P2'] = enc_p2.transform(df['CHAR_P2'])

X_cols = ['CHAR_P1', 'CHAR_P2', 'S_airborne_P1',
   'S_damage_P1', 'S_direction_P1', 'S_hit_stun_P1', 'S_position_x_P1',
   'S_position_y_P1', 'S_shield_P1', 'S_state_P1', 'S_state_age_P1',
   'S_stocks_P1', 'S_airborne_P2', 'S_damage_P2', 'S_direction_P2',
   'S_hit_stun_P2', 'S_position_x_P2', 'S_position_y_P2', 'S_shield_P2',
   'S_state_P2', 'S_state_age_P2', 'S_stocks_P2']

y_cols = ['B_damage_P1',
   'B_direction_P1', 'B_joystick_x_P1', 'B_joystick_y_P1',
   'B_position_x_P1', 'B_position_y_P1', 'B_cstick_x_P1', 'B_cstick_y_P1',
   'B_state_P1', 'B_raw_analog_P1', 'B_buttons_physical_P1',
   'B_triggers_physical_l_P1', 'B_triggers_physical_r_P1',
   'B_triggers_logical_P1']

df['S_airborne_P1'] = df['S_airborne_P1'].astype(bool).astype(float)
df['S_airborne_P2'] = df['S_airborne_P2'].astype(bool).astype(float)

X = df[X_cols]
y = df[y_cols]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [357]:
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)


y_train = y_scaler.transform(y_train)
y_test = y_scaler.transform(y_test)

In [358]:
class FrameDataset(Dataset):
    def __init__(self,X,y):
        self.x_train=torch.tensor(X,dtype=torch.float32)
        self.y_train=torch.tensor(y,dtype=torch.float32)
        
    def __len__(self):
        return len(self.y_train)
  
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]

In [359]:
dataset_train = FrameDataset(X_train, y_train)
dataset_test = FrameDataset(X_test, y_test)

In [360]:
#fix this later
train_loader = DataLoader(dataset_train,batch_size=16,shuffle=True)
test_loader = DataLoader(dataset_test,batch_size=16,shuffle=True)

In [361]:
class Net(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(Net, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
input_size = dataset.x_train.shape[1]
output_size = dataset.y_train.shape[1]
hidden_size = 128

epochs = 10

model = Net(input_size, output_size, hidden_size)

learning_rate = 0.005

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  
criterion = nn.MSELoss()

NameError: name 'dataset' is not defined

In [369]:
test_losses = []
for data, labels in test_loader:
    outputs = model(data)
    loss = criterion(outputs, labels)
    test_losses.append(loss.item())
np.average(test_losses)

1.0209653819136022

In [370]:
for epoch in range(epochs):
    train_losses = []
    for i, (data, labels) in enumerate(train_loader):

        optimizer.zero_grad()
        
        outputs = model(data)
        loss = criterion(outputs, labels)
        
        loss.backward()
        train_losses.append(loss.item())
        
        optimizer.step()
      
    test_losses = []
    
    for data, labels in test_loader:
        outputs = model(data)
        loss = criterion(outputs, labels)
        test_losses.append(loss.item())
        
    print('epoch {}, train loss {}'.format(epoch, np.average(train_losses)))
    print('epoch {}, test loss {}'.format(epoch, np.average(test_losses)))
    print(' ')
        

epoch 0, train loss 0.787378146298324
epoch 0, test loss 0.7733541371556563
 
epoch 1, train loss 0.768548049072676
epoch 1, test loss 0.7620000916817209
 


KeyboardInterrupt: 